In [1]:
import re
import string
import nltk

import pandas as pd 

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
file_name = "raw_tweets.csv"

data = pd.read_csv(file_name)

#### Remove retweets.

In [3]:
no_dup = data.drop_duplicates(subset=['tweet'])

#### Clean tweets following https://github.com/gracecarrillo/Political-Data-Science/blob/master/ETL_Sentiment_Analysis_Scotref2.ipynb.

In [4]:
len(data), len(no_dup)

(129249, 128925)

In [5]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jaint\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Remove special characters, links, hashtags, small words, and whitespace.

In [6]:
def process_tweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+', '', tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ')
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet

Remove punctuation and stopwords.

In [7]:
def text_process(tweet):
    no_punc = [char for char in list(tweet) if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    return [word for word in no_punc.lower().split() if word.lower() not in stop_words]

Use only word stems.

In [8]:
def stemming(tokens):
    stemmer = PorterStemmer()
    x = [stemmer.stem(w) for w in tokens]
    return ' '.join(x)

In [9]:
no_dup['tidy_tweet'] = no_dup.tweet.apply(process_tweet)
no_dup['tidy_tweet'] = no_dup.tidy_tweet.str.replace('[^a-zA-Z#]', ' ')

c:\users\jaint\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\jaint\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
no_dup['tokens'] = no_dup.tidy_tweet.apply(text_process)

c:\users\jaint\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
no_dup['stemmed'] = no_dup.tokens.apply(stemming)

c:\users\jaint\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Drop all tweets that end up with no tokens after cleaning. 

In [12]:
no_dup['token_length'] = no_dup.tokens.apply(lambda x: len(x))

c:\users\jaint\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
no_dup = no_dup[no_dup.token_length > 0]

In [14]:
no_dup = no_dup.drop(columns='token_length')

In [15]:
no_dup.to_csv('cleaned_tweets.csv', index=False)